In [1]:
### Running JupySQL and DuckDB
`!pip install jupysql duckdb-engine --quiet`


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
%load_ext sql

%sql duckdb://

Connecting to 'duckdb://'

In [10]:
%sql CREATE TABLE dim_date (	
[date] TEXT PRIMARY KEY,		
	[day] INT,						
	[suffix] VARCHAR(2),			
	[dow_name] VARCHAR(20),			
	[dow_num] INT,					
	[doy] INT,						
	[weekend] INT DEFAULT 0,		
	[woy] INT,						
	[fow] DATE,						
	[low] DATE,						
	[wom] INT,						
	[month] INT,					
	[moy_name] VARCHAR(20),			
	[fom] DATE,						
	[lom] DATE,						
	[gov_quarter] INT,				
	[year] INT,						
	[cy] INT,						
	[fy] INT,						
	[leap_year] INT	DEFAULT 0,		
	[53_weeks] INT DEFAULT 0,		
	[disp_date] VARCHAR(10),		
	[disp_mm_yyyy] VARCHAR(7),		
	[disp_mm_dd_yyyy] VARCHAR(10),	
	[disp_date_suffix] VARCHAR(8),	
	[ppid] INT,						
	[pp] VARCHAR(5),				
	[ppy] INT,						
	[ppn] INT,						
	[ppw] INT,						
	[ppd] INT,						
	[pp_start] DATE,				
	[pp_end] DATE,					
	[pay_day] INT DEFAULT 0,		
	[school_open] INT DEFAULT 0,	
	[school_day] INT DEFAULT 0,		
	[school_ab_day] VARCHAR(1)		
)

SyntaxError: invalid decimal literal (329021609.py, line 22)

In [6]:
SELECT * FROM dim_date

SyntaxError: invalid syntax (584836139.py, line 1)

In [6]:
import pandas as pd
from datetime import datetime, timedelta, date

# Define the date range
start_date = date(2023, 1, 1)
end_date = date(2070, 12, 31)

# School Days off and events
# (provided data)

# Initialize a DataFrame
columns = [
    'date', 'day', 'suffix', 'dow_name', 'dow_num', 'doy', 'weekend', 
    'woy', 'fow', 'low', 'wom', 'month', 'moy_name', 'fom', 'lom', 
    'gov_quarter', 'year', 'cy', 'fy', 'leap_year', '53_weeks', 
    'disp_date', 'disp_mm_yyyy', 'disp_mm_dd_yyyy', 'disp_date_suffix', 
    'ppid', 'pp', 'ppy', 'ppn', 'ppw', 'ppd', 'pp_start', 'pp_end', 
    'pay_day', 'school_open', 'school_day', 'school_ab_day'
]
df = pd.DataFrame(columns=columns)

# Helper function to get ordinal suffix
def ordinal(n):
    if 10 <= n % 100 <= 20:
        suffix = 'th'
    else:
        suffix = {1: 'st', 2: 'nd', 3: 'rd'}.get(n % 10, 'th')
    return suffix

# Helper function to get pay period info based on date
def get_pay_period(d):
    pp_start_date = date(2023, 1, 1)  # Pay period 23-01 start date
    pp_num = 1  # Starting pay period number
    pp_year = 23  # Starting pay period year
    while pp_start_date <= d:
        pp_end_date = pp_start_date + timedelta(days=13)  # Pay periods are 2 weeks long
        if pp_end_date > d:
            break
        pp_start_date = pp_end_date + timedelta(days=1)
        pp_num += 1
        if pp_num > 26:  # Reset pay period number and increment year
            pp_num = 1
            pp_year += 1
    return pp_start_date, pp_end_date, pp_num, pp_year

# Iterate through the date range and populate the DataFrame
current_date = start_date
ab_day_flag = 'A'  # Starting with 'A' on 2023-09-07
while current_date <= end_date:
    # Common date attributes
    day_of_week = current_date.weekday() + 1  # 1 = Sunday, 2 = Monday, etc.
    weekend = 1 if day_of_week in [1, 7] else 0
    month = current_date.month
    year = current_date.year
    gov_quarter = (month - 1) // 3 + 1
    week_of_year = current_date.isocalendar()[1]
    week_start = current_date - timedelta(days=current_date.weekday())
    week_end = week_start + timedelta(days=6)
    leap_year = 1 if year % 4 == 0 and (year % 100 != 0 or year % 400 == 0) else 0
    # Pay period attributes
    pp_start, pp_end, pp_num, pp_year = get_pay_period(current_date)
    # School day attributes
    school_open = 1 if (datetime(2023, 9, 7).date() <= current_date <= datetime(2024, 7, 1).date()) and not weekend else 0
    if school_open and current_date not in [d[0] for d in school_days_off]:
        school_day = 1
        school_ab_day = ab_day_flag
        ab_day_flag = 'B' if ab_day_flag == 'A' else 'A'  # Alternate A/B days
    else:
        school_day = 0
        school_ab_day = None
    # Append row to DataFrame
    df = df.append({
        'date': current_date,
        'day': current_date.day,
        'suffix': ordinal(current_date.day),
        'dow_name': current_date.strftime('%A'),
        'dow_num': day_of_week,
        'doy': current_date.timetuple().tm_yday,
        'weekend': weekend,
        'woy': week_of_year,
        'fow': week_start,
        'low': week_end,
        'wom': (current_date.day - 1) // 7 + 1,
        'month': month,
        'moy_name': current_date.strftime('%B'),
        'fom': current_date.replace(day=1),
        'lom': (current_date.replace(month=current_date.month % 12 + 1, day=1) - timedelta(days=1)),
        'gov_quarter': gov_quarter,
        'year': year,
        'cy': year,
        'fy': year,
        'leap_year': leap_year,
        '53_weeks': 1 if week_of_year == 53 else 0,
        'disp_date': current_date.strftime('%Y-%m-%d'),
        'disp_mm_yyyy': current_date.strftime('%m/%Y'),
        'disp_mm_dd_yyyy': current_date.strftime('%m/%d/%Y'),
        'disp_date_suffix': current_date.strftime('%Y%m%d'),
        'ppid': pp_year * 100 + pp_num,
        'pp': f'{pp_year}-{str(pp_num).zfill(2)}',
        'ppy': pp_year,
        'ppn': pp_num,
        'ppw': 1 if (current_date - pp_start).days < 7 else 2,
        'ppd': (current_date - pp_start).days + 1,
        'pp_start': pp_start,
        'pp_end': pp_end,
        'pay_day': 0,  # Additional logic may be required
        'school_open': school_open,
        'school_day': school_day,
        'school_ab_day': school_ab_day
    }, ignore_index=True)
    # Move to next date
    current_date += timedelta(days=1)

# Output the DataFrame
print(df)


ModuleNotFoundError: No module named 'pandas'

# Output

## Becky's data may look like this:

```python
becky_data = 
[
    ('2023-05-18 08:00:00-0400', 'pain-meds oxycodone-15.mg',      'location:abdomen_lower-right-quadrant, level:6'),
    ('2023-05-18 09:30:00-0400', 'bowel-movement diarrhea',       'liquid'),
    ('2023-05-18 11:15:00-0400', 'morning-meds group__becky-morning-meds', None),
    ('2023-05-18 13:45:00-0400', 'bowel-movement diarrhea',       'diced'),
    ('2023-05-18 16:00:00-0400', 'pain-meds - oxycodone-15.mg',     'location:abdomen_lower-right-quadrant, level:5')
]
```

## Fitzy's data may look like this:

```python
fitzy_data = 
[
    ('2023-05-18 07:14:00-0400', 'brush-teeth',                     None),
    ('2023-05-18 07:15:00-0400', 'daily-vitamins',                  None),
    ('2023-05-18 16:36:00-0400', 'homework-completed',              None),
    ('2023-05-18 18:00:00-0400', 'baseball-practice',               None),
    ('2023-05-18 21:39:00-0400', 'brush-teeth',                     None)
]
```

## Adge's data may look like this:

```python
adge_data = [
    ('2023-05-18 05:14:00-0400', 'morning-meds group__adge-morning-meds',   None),
    ('2023-05-18 08:15:00-0400', 'void-bladder',                            'sudden'),
    ('2023-05-18 08:36:00-0400', 'void-bladder',                            'sudden'),
    ('2023-05-18 18:00:00-0400', 'afternoon-meds group__adge-afternoon-meds', None),
    ('2023-05-18 21:39:00-0400', 'walk',                                    '1.2 miles')
]
```

# Create User Profile Table

In [ ]:
CREATE TABLE
    profile (
        id INT NOT NULL PRIMARY KEY IDENTITY(1,1),      -- unique, primary key
        user_id INT NOT NULL,                           -- foreign key to users table
        full_name VARCHAR(255),                         -- full name of the user, as used in official documents and reports
        name VARCHAR(255),                              -- name of the user, as used in informal settings
        email VARCHAR(255),                             -- email address of the user
        FOREIGN KEY (user_id) REFERENCES users(id)
    )

# Tracking Medications

The code below is a rudamentary example of the tables required to implement the tracking people taking medicaitons. This is only one possible solution of many.

In [ ]:
CREATE TABLE 
    medications (
        id INT NOT NULL PRIMARY KEY IDENTITY(1,1),      -- unique, primary key
        medication VARCHAR(100),                        -- our common name for this medication
        generic_name VARCHAR(255),                      -- the generic name for this medication
        brand_name VARCHAR(255),                        -- the brand name for this medication
        description VARCHAR(255),                       -- a description of this medication
    )

CREATE TABLE
    medication_stubs (
        id INT NOT NULL PRIMARY KEY IDENTITY(1,1),
        stub VARCHAR(100) NOT NULL,                     -- text-based unique identifier, easy to read and understand 
        medication_id INT NOT NULL,                     -- foreign key to medications table
        medication_dosage_id INT NOT NULL,              -- foreign key to medication_dosages table
        FOREIGN KEY (medication_id) REFERENCES medications(id),
        FOREIGN KEY (medication_dosage_id) REFERENCES medication_dosages(id)
    )

CREATE TABLE
    medication_availability (
        id INT NOT NULL PRIMARY KEY IDENTITY(1,1),      -- unique, primary key
        medication_id INT NOT NULL,                     -- foreign key to medications table
        medication_availability VARCHAR(255),           -- how the medication is available - prescription, otc, etc.
    )

CREATE TABLE
    medication_dosages (
        id INT NOT NULL PRIMARY KEY IDENTITY(1,1),      -- unique, primary key
        medication_id INT NOT NULL,                     -- foreign key to medications table
        dosage VARCHAR(255),                            -- the dosage of the medication
        dosage_unit VARCHAR(255),                       -- the unit of the dosage
        dosage_frequency VARCHAR(255),                  -- how often the dosage is taken
        dosage_duration FLOAT,                          -- number of hours between dosages
        FOREIGN KEY (medication_id) REFERENCES medications(id)
    )

CREATE TABLE
    medication_side_effects (
        id INT NOT NULL PRIMARY KEY IDENTITY(1,1),      -- unique, primary key
        medication_id INT NOT NULL,                     -- foreign key to medications table
        side_effect_id INT NOT NULL,                    -- foreign key to side_effects table
        FOREIGN KEY (medication_id) REFERENCES medications(id),
        FOREIGN KEY (side_effect_id) REFERENCES side_effects(id)
    )

CREATE TABLE
    side_effects (
        id INT NOT NULL PRIMARY KEY IDENTITY(1,1),      -- unique, primary key
        side_effect VARCHAR(255),                       -- the side effect
        additional_information VARCHAR(255),            -- additional information about the side effect
        seriousness VARCHAR(255),                       -- how serious the side effect is
        duration VARCHAR(255),                          -- how long the side effect lasts
        overdose_warning INT                            -- whether or not this side effect is a warning sign of overdose
    )

CREATE TABLE
    prescriptions (
        id INT NOT NULL PRIMARY KEY IDENTITY(1,1),      -- unique, primary key
        medication_stub_id INT NOT NULL,                -- foreign key to medication_stubs table
        physician_id INT NOT NULL,                      -- foreign key to physicians table
    )

CREATE VIEW
    list_of_medication_stubs_w_medication_info (
        SELECT
            s.stub,                                     -- [medication_stubs].[stub]                            -- our unique identifier for this stub
            m.medication,                               -- [medications].[medication]                           -- our common name for this medication
            m.generic_name,                             -- [medications].[generic_name]                         -- the generic name for this medication
            m.brand_name,                               -- [medications].[brand_name]                           -- the brand name for this medication
            m.medication_availability,                  -- [medication_availability].[medication_availability]  -- how the medication is available - prescription, otc, etc.
            d.dosage,                                   -- [medication_dosages].[dosage]                        -- the dosage of the medication
            d.dosage_unit,                              -- [medication_dosages].[dosage_unit]                   -- the unit of the dosage
            d.dosage_frequency,                         -- [medication_dosages].[dosage_frequency]              -- how often the dosage is taken
            d.dosage_duration                           -- [medication_dosages].[dosage_duration]               -- number of hours between dosages
        FROM medication_stubs s 
            INNER JOIN medications m ON s.medication_id = m.id
            INNER JOIN medication_dosages d ON s.medication_dosage_id = d.id
    )
    


# General Medical Tables   

In [ ]:


CREATE TABLE
    physicians (
        id INT NOT NULL PRIMARY KEY IDENTITY(1,1),      -- unique, primary key
        name_first VARCHAR(255),                        -- first name of the physician
        name_last VARCHAR(255),                         -- last name of the physician
        name_middle VARCHAR(255),                       -- middle name of the physician
        name_suffix VARCHAR(255),                       -- suffix of the physician
        specialty_id VARCHAR(255),                      -- the specialty of the physician
        address VARCHAR(255),                           -- the address of the physician
        city VARCHAR(255),                              -- the city of the physician
        state VARCHAR(255),                             -- the state of the physician
        zip_code VARCHAR(255),                          -- the zip code of the physician
        phone VARCHAR(255),                             -- the phone number of the physician
        fax VARCHAR(255),                               -- the fax number of the physician
        email VARCHAR(255),                             -- the email address of the physician
        website VARCHAR(255),                           -- the website of the physician
        FOREIGN KEY (specialty_id) REFERENCES specialty(id)
    )

CREATE TABLE
    medical_speciality (
        id INT NOT NULL PRIMARY KEY IDENTITY(1,1),      -- unique, primary key
        specialty VARCHAR(255),                         -- the specialty of the physician
        description VARCHAR(255),                       -- a description of the specialty
    )

CREATE TABLE
    conditions (
        id INT NOT NULL PRIMARY KEY IDENTITY(1,1),      -- unique, primary key
        condition VARCHAR(255),                         -- the condition
        description VARCHAR(255),                       -- a description of the condition
    )

CREATE TABLE
    physician_conditions(
        id INT NOT NULL PRIMARY KEY IDENTITY(1,1),      -- unique, primary key
        physician_id INT NOT NULL,                      -- foreign key to physicians table
        condition_id INT NOT NULL,                      -- foreign key to conditions table
        FOREIGN KEY (physician_id) REFERENCES physicians(id),
        FOREIGN KEY (condition_id) REFERENCES conditions(id)
    )

CREATE TABLE
    condition_side_effects (
        id INT NOT NULL PRIMARY KEY IDENTITY(1,1),      -- unique, primary key
        condition_id INT NOT NULL,                      -- foreign key to conditions table
        side_effect_id INT NOT NULL,                    -- foreign key to side_effects table
        FOREIGN KEY (condition_id) REFERENCES conditions(id),
        FOREIGN KEY (side_effect_id) REFERENCES side_effects(id)
    )
    

In [ ]:
-- Example of a stub: "tylenol-1000-mg"
-- Medication Stubs are used in tracking the medications people take.
-- A stub is a text-based unique identifier that is easy to read and understand.
-- It maps to a specific medication, amount, and unit of measurement.

In [3]:
medication01 = "oxycodone-7.5-mg"
medication02 = "oxycodone-15-mg"
medication03 = "oxycodone-30-mg"
medication04 = "lorazapam-0.5-mg"
medication05 = "venlafaxine-er-150-mg"
medication06 = "bupropion-xr-300-mg"
medication07 = "clonazepam-1-mg"
medication08 = "omeprazole-30-mg"
medication09 = "tamsulosin-0.4-mg"
medication10 = "vyvanse-70-mg"

# to query all of a stub's information, follow the example below
sql = "select * from list_of_medication_stubs_w_medication_info where stub = 'bupropion-xr-300-mg'"

# medications table fields: id,                             -- 1
#                           stub,                           -- bupropion-xr-300-mg
#                           med.id,                         -- 18
#                           med.name,                       -- bupropion xr
#                           med.description,                -- lorem ipsum solor sit amet
#                           med.format,                     -- tablet_xr
#                           med.units,                      -- mg
#                           med.strength-per-unit,          -- 300
#                           med.type,                       -- prescription
#                           med.additional-information-url, -- https://www.drugs.com/bupropion.html
#                           created_at,                     -- 2021-05-18 08:00:00-0400
#                           updated_at                      -- 2021-05-18 08:00:00-0400

## Groups

In [ ]:
group__adge_morning_meds    = ["venlafaxine-150-mg", "clonazepam-1-mg", "bupropion-xr-300-mg", "omeprazole-30-mg", "tamsulosin-0.4-mg", "vyvanse-70-mg"]
group__adge_afternoon_meds  = ["venlafaxine-150-mg", "clonazepam-1-mg"]
adge_med_groups = [group__adge_morning_meds, group__adge_afternoon_meds]

## Example Data

In [2]:
# Becky's Data 

date_times = [
    "2023-05-18 08:00:00-0400",
    "2023-05-18 09:30:00-0400",
    "2023-05-18 11:15:00-0400",
    "2023-05-18 13:45:00-0400",
    "2023-05-18 16:00:00-0400"
]

lines = [
    "pain-meds - oxycodone 15 mg",
    "bowel-movement - diarrhea",
    "morning-meds (see group)",
    "bowel-movement - diarrhea",
    "pain-meds - oxycodone 15 mg"
]

details = [
    "location:abdomen_lower-right-quadrant, level:6",
    "liquid",
    None,
    "diced",
    "location:abdomen_lower-right-quadrant, level:5"
]

combined_list = list(zip(date_times, lines, details))

print("Becky's data may look like this:")
for item in combined_list:
    print(item)

# Fitzy's Data 

date_times = [
    "2023-05-18 07:14:00-0400",
    "2023-05-18 07:15:00-0400",
    "2023-05-18 16:36:00-0400",
    "2023-05-18 18:00:00-0400",
    "2023-05-18 21:39:00-0400"
]

lines = [
    "brush-teeth",
    "daily-vitamins",
    "homework-completed",
    "baseball-practice",
    "brush-teeth"
]

details = [
    None,
    None,
    None,
    None,
    None
]

combined_list = list(zip(date_times, lines, details))

print("Fitzy's data may look like this:")
for item in combined_list:
    print(item)

# Adge's Data 

date_times = [
    "2023-05-18 05:14:00-0400",
    "2023-05-18 08:15:00-0400",
    "2023-05-18 08:36:00-0400",
    "2023-05-18 18:00:00-0400",
    "2023-05-18 21:39:00-0400"
]

lines = [
    "morning-meds",
    "void-bladder",
    "void-bladder",
    "afternoon-meds",
    "walk"
]

details = [
    None,
    "sudden",
    "sudden",
    None,
    "1.2 miles"
]

combined_list = list(zip(date_times, lines, details))

print("Adge's data may look like this:")
for item in combined_list:
    print(item)

Becky's data may look like this:
('2023-05-18 08:00:00-0400', 'pain-meds - oxycodone 15 mg', 'location:abdomen_lower-right-quadrant, level:6')
('2023-05-18 09:30:00-0400', 'bowel-movement - diarrhea', 'liquid')
('2023-05-18 11:15:00-0400', 'morning-meds (see group)', None)
('2023-05-18 13:45:00-0400', 'bowel-movement - diarrhea', 'diced')
('2023-05-18 16:00:00-0400', 'pain-meds - oxycodone 15 mg', 'location:abdomen_lower-right-quadrant, level:5')
Fitzy's data may look like this:
('2023-05-18 07:14:00-0400', 'brush-teeth', None)
('2023-05-18 07:15:00-0400', 'daily-vitamins', None)
('2023-05-18 16:36:00-0400', 'homework-completed', None)
('2023-05-18 18:00:00-0400', 'baseball-practice', None)
('2023-05-18 21:39:00-0400', 'brush-teeth', None)
Adge's data may look like this:
('2023-05-18 05:14:00-0400', 'morning-meds', None)
('2023-05-18 08:15:00-0400', 'void-bladder', 'sudden')
('2023-05-18 08:36:00-0400', 'void-bladder', 'sudden')
('2023-05-18 18:00:00-0400', 'afternoon-meds', None)
('20